# OPM Data Separation Analysis
<i><b>
Christopher Boomhower<sub>1</sub>, Stacey Fabricant<sub>2</sub>, Alex Frye<sub>1</sub>, David Mumford<sub>2</sub>, Michael Smith<sub>1</sub>, Lindsay Vitovsky<sub>1</sub>

<sub>1</sub> Southern Methodis Univeristy, Dallas, TX, US
<sub>2</sub> Penn Mutual Life Insurance Co, Horsham PA
</i></b>


### Introduction

background text...

**our intent is to: 1)..2)...3)........**

### Data Understanding

Data Source Background Text & citation links

Dataset Attribute Descriptions
(e.g. 1. **tripduration** - *Integer* - The total tim....)


### Load the Data

text intro

In [ ]:
## Import libraries
import pickle
import os
import glob
import pandas as pd
import numpy as np
from IPython.display import display
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import json
import missingno as msno
import prettytable

## Library Options

pd.options.mode.chained_assignment = None

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
## Pre-defined Functions for use later
def pickleObject(objectname, filename, filepath = "PickleJar/"):
    fullpicklepath = "{0}{1}.pkl".format(filepath, filename)
    # Create a variable to pickle and open it in write mode
    picklefile = open(fullpicklepath, 'wb')
    pickle.dump(objectname, picklefile)
    picklefile.close()
    
def unpickleObject(filename, filepath = "PickleJar/"):
    fullunpicklepath = "{0}{1}.pkl".format(filepath, filename)
    # Create an variable to pickle and open it in write mode
    unpicklefile = open(fullunpicklepath, 'rb')
    unpickleObject = pickle.load(unpicklefile)
    unpicklefile.close()

    return unpickleObject
    
def clear_display():
    from IPython.display import clear_output
    
## Pre-defined variables for use later
dataOPMPath = "dataOPM"
dataEMPPath = "dataEMP"
PickleJarPath = "PickleJar"

In [ ]:
%%time

## Load OPMSeparation Files

OPMDataFiles = glob.glob(os.path.join(dataOPMPath, "*.txt"))

for i in range(0,len(OPMDataFiles)):
    OPMDataFiles[i] = OPMDataFiles[i].replace("\\","/")

OPMDataList = []

for i,j in zip(OPMDataFiles,range(0,len(OPMDataFiles))):
    OPMDataList.append(pd.read_csv(i, dtype = 'str'))
    display(OPMDataList[j].head())

## Load the SEPDATA_FY2015 file into it's own object
indexes = [i for i,x in enumerate(OPMDataFiles) if x == 'dataOPM/SEPDATA_FY2015.txt']
OPMDataOrig = OPMDataList[indexes[0]]


In [ ]:
%%time

#print(OPMDataFiles)

print(len(OPMDataOrig))

##### Merge / Modify Codes / Aggregate Attributes to be more descriptive per the metadata files

OPMDataMerged = OPMDataOrig.copy()

##AGYSUB - AGYTYP, AGY
indexes = [i for i,x in enumerate(OPMDataFiles) if x == 'dataOPM/DTagy.txt']
OPMDataMerged = OPMDataMerged.merge(OPMDataList[indexes[0]], on = 'AGYSUB', how = 'left')

##EFDate - quarter, month
indexes = [i for i,x in enumerate(OPMDataFiles) if x == 'dataOPM/DTefdate.txt']
OPMDataMerged = OPMDataMerged.merge(OPMDataList[indexes[0]], on = 'EFDATE', how = 'left')

##AGELVL - AGELVLT
indexes = [i for i,x in enumerate(OPMDataFiles) if x == 'dataOPM/DTagelvl.txt']
OPMDataMerged = OPMDataMerged.merge(OPMDataList[indexes[0]], on = 'AGELVL', how = 'left')

##LOSLVL - LOSLVLT
indexes = [i for i,x in enumerate(OPMDataFiles) if x == 'dataOPM/DTloslvl.txt']
OPMDataMerged = OPMDataMerged.merge(OPMDataList[indexes[0]], on = 'LOSLVL', how = 'left')

##LOC - LocTypeT, LocT
indexes = [i for i,x in enumerate(OPMDataFiles) if x == 'dataOPM/DTloc.txt']
OPMDataMerged = OPMDataMerged.merge(OPMDataList[indexes[0]], on = 'LOC', how = 'left')

##OCC - OCCTYPT, OCCFAM
indexes = [i for i,x in enumerate(OPMDataFiles) if x == 'dataOPM/DTocc.txt']
OPMDataMerged = OPMDataMerged.merge(OPMDataList[indexes[0]], on = 'OCC', how = 'left')

##PATCO - PATCOT
indexes = [i for i,x in enumerate(OPMDataFiles) if x == 'dataOPM/DTpatco.txt']
OPMDataMerged = OPMDataMerged.merge(OPMDataList[indexes[0]], on = 'PATCO', how = 'left')

##PPGRD - PayPlan, PPGroup, PPTYP
indexes = [i for i,x in enumerate(OPMDataFiles) if x == 'dataOPM/DTppgrd.txt']
OPMDataMerged = OPMDataMerged.merge(OPMDataList[indexes[0]], on = 'PPGRD', how = 'left')

##SALLVL - SALLVLT
indexes = [i for i,x in enumerate(OPMDataFiles) if x == 'dataOPM/DTsallvl.txt']
OPMDataMerged = OPMDataMerged.merge(OPMDataList[indexes[0]], on = 'SALLVL', how = 'left')

##TOA - TOATYP
indexes = [i for i,x in enumerate(OPMDataFiles) if x == 'dataOPM/DTtoa.txt']
OPMDataMerged = OPMDataMerged.merge(OPMDataList[indexes[0]], on = 'TOA', how = 'left')

##WORKSCH - WSTYPT
indexes = [i for i,x in enumerate(OPMDataFiles) if x == 'dataOPM/DTwrksch.txt']
OPMDataMerged = OPMDataMerged.merge(OPMDataList[indexes[0]], on = 'WORKSCH', how = 'left')

OPMDataMerged["SALARY"] = OPMDataMerged["SALARY"].apply(pd.to_numeric)
display(OPMDataMerged.head())
#del OPMDataList,OPMDataFiles

In [ ]:
%%time

if os.path.isfile(PickleJarPath+"/EMPDataOrig4Q.pkl"):
    print("Found the File! Loading Pickle Now!")
    EMPDataOrig4Q = unpickleObject("EMPDataOrig4Q")
else:
    ## Load EMPData Files

    indexes = []
    EMPDataFiles = []
    EMPDataList = []
    EMPDataOrig = []

    for i,qtr in enumerate(["Q1", "Q2", "Q3", "Q4"]): 
        EMPDataFiles.append(glob.glob(os.path.join(dataEMPPath, qtr + "/*.txt")))

        for j in range(0,len(EMPDataFiles[i])):
            EMPDataFiles[i][j] = EMPDataFiles[i][j].replace("\\","/")

        EMPDataList.append([])

        for j,file in enumerate(EMPDataFiles[i]):
            EMPDataList[i].append(pd.read_csv(file, dtype = 'str'))
            if i == 0:
                display(EMPDataList[i][j].head())

        ## Load the FactData files into it's own object
        indexes.append([])
            ##[qtr][fileindex from EMPDataList]
        indexes[i]=[j for j,x in enumerate(EMPDataFiles[i]) if dataEMPPath + '/' + qtr + '/FACTDATA' in x]   

        EMPDataOrig.append([])

        EMPDataOrig[i] = pd.concat([EMPDataList[i][indexes[i][j]] for j in range(0,len(indexes[i]))]) 
        EMPDataOrig[i]["QTR"] = str(i+1)

            ## modify data type for salary for aggregation
        EMPDataOrig[i]["SALARY"] = EMPDataOrig[i]["SALARY"].str.replace(',', '').str.replace('$', '').str.replace(' ', '').apply(pd.to_numeric)

            ## Load Metadata
        ##AGYSUB - AGYTYP, AGY
        ind2 = [i for i,x in enumerate(EMPDataFiles[i]) if x == dataEMPPath + '/' + qtr + '/DTagy.txt']
        EMPDataOrig[i] = EMPDataOrig[i].merge(EMPDataList[i][ind2[0]], on = 'AGYSUB', how = 'left')

        ##AGELVL - AGELVLT
        ind2 = [i for i,x in enumerate(EMPDataFiles[i]) if x == dataEMPPath + '/' + qtr + '/DTagelvl.txt']
        EMPDataOrig[i] = EMPDataOrig[i].merge(EMPDataList[i][ind2[0]], on = 'AGELVL', how = 'left')

        ##LOSLVL - LOSLVLT
        ind2 = [i for i,x in enumerate(EMPDataFiles[i]) if x == dataEMPPath + '/' + qtr + '/DTloslvl.txt']
        EMPDataOrig[i] = EMPDataOrig[i].merge(EMPDataList[i][ind2[0]], on = 'LOSLVL', how = 'left')

        ##LOC - LocTypeT, LocT
        ind2 = [i for i,x in enumerate(EMPDataFiles[i]) if x == dataEMPPath + '/' + qtr + '/DTloc.txt']
        EMPDataOrig[i] = EMPDataOrig[i].merge(EMPDataList[i][ind2[0]], on = 'LOC', how = 'left')

        ##OCC - OCCTYPT, OCCFAM
        ind2 = [i for i,x in enumerate(EMPDataFiles[i]) if x == dataEMPPath + '/' + qtr + '/DTocc.txt']
        EMPDataOrig[i] = EMPDataOrig[i].merge(EMPDataList[i][ind2[0]], on = 'OCC', how = 'left')

        ##PATCO - PATCOT
        ind2 = [i for i,x in enumerate(EMPDataFiles[i]) if x == dataEMPPath + '/' + qtr + '/DTpatco.txt']
        EMPDataOrig[i] = EMPDataOrig[i].merge(EMPDataList[i][ind2[0]], on = 'PATCO', how = 'left')

        ##PPGRD - PayPlan, PPGroup, PPTYP
        ind2 = [i for i,x in enumerate(EMPDataFiles[i]) if x == dataEMPPath + '/' + qtr + '/DTppgrd.txt']
        EMPDataOrig[i] = EMPDataOrig[i].merge(EMPDataList[i][ind2[0]], on = 'PPGRD', how = 'left')

        ##SALLVL - SALLVLT
        ind2 = [i for i,x in enumerate(EMPDataFiles[i]) if x == dataEMPPath + '/' + qtr + '/DTsallvl.txt']
        EMPDataOrig[i] = EMPDataOrig[i].merge(EMPDataList[i][ind2[0]], on = 'SALLVL', how = 'left')

        ##TOA - TOATYP
        ind2 = [i for i,x in enumerate(EMPDataFiles[i]) if x == dataEMPPath + '/' + qtr + '/DTtoa.txt']
        EMPDataOrig[i] = EMPDataOrig[i].merge(EMPDataList[i][ind2[0]], on = 'TOA', how = 'left')

        ##WORKSCH - WSTYPT
        ind2 = [i for i,x in enumerate(EMPDataFiles[i]) if x == dataEMPPath + '/' + qtr + '/DTwrksch.txt']
        EMPDataOrig[i] = EMPDataOrig[i].merge(EMPDataList[i][ind2[0]], on = 'WORKSCH', how = 'left')

        display(EMPDataOrig[i].head())

    EMPDataOrig4Q = pd.concat([EMPDataOrig[j] for j in range(0,len(EMPDataOrig))])
    pickleObject(EMPDataOrig4Q, "EMPDataOrig4Q")

print(len(EMPDataOrig4Q))

In [ ]:
display(EMPDataOrig4Q.describe().transpose())

In [ ]:
%matplotlib inline

sns.boxplot(y = "SALARY", data = EMPDataOrig4Q)
#sns.boxplot(y = "QTR", data = EMPDataOrig4Q)

In [ ]:
%%time
%matplotlib inline

##Aggregate Number of Total Separations in current month for given Occ
Agg = pd.DataFrame({'SEPCount_EFDATE_OCC' : OPMDataMerged.groupby(["EFDATE", "OCC"]).size()}).reset_index()
display(Agg.head())
OPMDataMerged = OPMDataMerged.merge(Agg, on = ['EFDATE','OCC'], how = 'left')
EMPDataOrig4Q = EMPDataOrig4Q.merge(Agg, left_on = ['DATECODE','OCC'], right_on = ['EFDATE','OCC'], how = 'left')

##Aggregate Number of Total Separations in current month for given LOC
Agg = pd.DataFrame({'SEPCount_EFDATE_LOC' : OPMDataMerged.groupby(["EFDATE", "LOC"]).size()}).reset_index()
display(Agg.head())
OPMDataMerged = OPMDataMerged.merge(Agg, on = ['EFDATE','LOC'], how = 'left')
EMPDataOrig4Q = EMPDataOrig4Q.merge(Agg, left_on = ['DATECODE','LOC'], right_on = ['EFDATE','LOC'], how = 'left')

##Average Quarterly EMP Salary by occ 
Agg = pd.DataFrame({'count' : EMPDataOrig4Q.groupby(["QTR", "OCC"]).size()}).reset_index()
Agg2 = pd.DataFrame({'IndSalarySum' : EMPDataOrig4Q.groupby(["QTR", "OCC"])["SALARY"].sum()}).reset_index()
Agg = Agg.merge(Agg2,on=["QTR", "OCC"])
Agg["IndAvgSalary"] = Agg["IndSalarySum"]/Agg["count"]

del Agg["count"]
del Agg["IndSalarySum"]

display(Agg.head())

OPMDataMerged = OPMDataMerged.merge(Agg, on = ['QTR','OCC'], how = 'left')
EMPDataOrig4Q = EMPDataOrig4Q.merge(Agg, on = ['QTR','OCC'], how = 'left')


display(OPMDataMerged.head())
display(EMPDataOrig4Q.head())

In [ ]:
display(OPMDataMerged.tail())
display(EMPDataOrig4Q.tail())

In [ ]:
%%time

#### Analyze Missing Values
#use missingno here
filtered_msnoData = msno.nullity_filter(OPMDataMerged, filter='bottom', n=10, p=0.999)
msno.matrix(filtered_msnoData.sample(50000))

filtered_msnoData = msno.nullity_filter(EMPDataOrig4Q, filter='bottom', n=10, p=0.999)
msno.matrix(filtered_msnoData.sample(50000))

del filtered_msnoData
#### Fix Data Types

In [ ]:
%%time

display(OPMDataMerged.describe().transpose())

### Pull Bureau of Labor Statistics data

In [ ]:
%%time

def bls(series, start, end):
    headers = {'Content-type': 'application/json'}
    data = json.dumps({"seriesid": series,
                       "startyear":start,
                       "endyear":end,
                       "catalog":False,
                       "calculations":False,
                       "annualaverage":False,
                       "registrationkey":"7a89c8d7979349fba8914b8be16a1646"})
    
    p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)
    json_data = json.loads(p.text)
    bls = []
    for series in json_data['Results']['series']:
        #x=prettytable.PrettyTable(["series id","year","period","value","footnotes"])
        result = pd.DataFrame(columns=["series id","year","period","value","footnotes"])
        seriesId = series['seriesID']
        for item in series['data']:
            year = item['year']
            period = item['period']
            value = item['value']
            footnotes=""
            for footnote in item['footnotes']:
                if footnote:
                    footnotes = footnotes + footnote['text'] + ','
            if 'M01' <= period <= 'M12':
                #x.add_row([seriesId,year,period,value,footnotes[0:-1]])
                y = pd.DataFrame({"series id" : seriesId,
                                  "year" : year,
                                  "period" : period,
                                  "value" : value,
                                  "footnotes" : footnotes}, index = [0])
                result = result.append(y, ignore_index = True)
                #x.concat([seriesId,year,period,value,footnotes[0:-1]], ignore_index=True)
        #result = x.append(y)
        bls.append(result)
    return(bls)

In [ ]:
#test=pd.DataFrame(columns=["series id","year","period","value","footnotes"])
##test2 = pd.DataFrame([3,2012,'M12',132,'dude'], columns=["series id","year","period","value","footnotes"]).T
#test2 = pd.DataFrame({"series id" : 3, "year" : 2012, "period" : 'M12', "value" : 132, "footnotes" : 'dude'}, index = [0])
#test3 = pd.DataFrame({"series id" : 4, "year" : 2011, "period" : 'M11', "value" : 135, "footnotes" : 'dude2'}, index = [0])
##result = pd.append([test,test2], axis = 1)
#result = test.append([test2, test3], ignore_index = True)
#display(result)

In [ ]:
%%time

# Pull job openings and labor turnover data
JTL = bls(['JTU91000000HIL',
     'JTU91000000JOL',
     'JTU91000000LDL',
     'JTU91000000OSL',
     'JTU91000000QUL',
     'JTU91000000TSL',
     'JTU91000000UNL',
     'JTU91000000UOL'], "2014", "2015")

display(JTL[3])